# Intersection product on the cartesian product of a curve

* Authors: Daniele Agostini, Daniel Plaumann, Rainer Sinn, Yannik Wesner.
* Version: OSCAR version 1.1.0 or newer. 
* Reference: [APSW] D. Agostini, D. Plaumann, R. Sinn, Y. Wesner, *Plane quartics and heptagons*, arXiv:2408.15759 (2024). 

We demonstrate how to perform intersection calculus on the cartesian product of a smooth curve.

Let $C$ be a smooth and irreducible projective complex curve of genus $g$. We consider the cartesian product $C^n$ for $n>0$. For any indexes $1\leq i < j \leq n$ we consider the projections
$$ \operatorname{pr}_i : C^n \to C, \quad \operatorname{pr}_{ij}: C^n \to C\times C $$
onto the $i$-th and the $(i,j)$-th components respectively. These induce some natural numerical equivalence classes of divisors on $C^n$:

* $x_i := \operatorname{pr}_i^*(x)$ where $x\in C$ is any point.
* $\Delta_{ij} := \operatorname{pr}^*(\Delta_{ij})$ where $\Delta\subseteq C\times C$ is the diagonal.

Many problems in the enumerative geometry of $C$ can be phrased in terms of intersection calculus of these classes. We aim to perform these computationally in OSCAR. We first set up a ring with the variables that we need

In [ ]:
using Oscar
using Graphs

  ___   ____   ____    _    ____
 / _ \ / ___| / ___|  / \  |  _ \   |  Combining ANTIC, GAP, Polymake, Singular
| | | |\___ \| |     / _ \ | |_) |  |  Type "?Oscar" for more information
| |_| | ___) | |___ / ___ \|  _ <   |  Manual: https://docs.oscar-system.org
 \___/ |____/ \____/_/   \_\_| \_\  |  1.2.0-DEV #master 2941518 2024-07-16


In [ ]:
# We set the genus g of the curve C and the number of copies n in the Cartesian product C^n
n = 7
g = 3

In [ ]:
# We define the polynomial ring that we use to describe the intersection product: 
# it has variables x[1],...,x[n] and Delta[1,2],...,Delta[n-1,n]

RCart, x, Delta = polynomial_ring(QQ, "x" => (1:n), "Delta" => (1:n, 1:n));

# We are going to redefine it later in the examples

## Reduced form

In order to compute intersection numbers, we need to consider monomials in the classes $x_i$ and $\Delta_{ij}$. The first observation is that these monomials can be brought into reduced form via three relations: the first one is

**Relation 1**: $x_i^2 = 0$ for all $1\leq i \leq n$

This relation allows us to discard all monomials where one variable $x_i$ appears with exponent at least two.

This is implemented by the following function: 

In [2]:
# The following function implements relation 1: x[i]^2 = 0

function reduce_term_relation_1(m)
if m==0
    return m
else
    mred = m;
    for i in 1:n
        if exponent(mred,1,var_index(x[i]))>1
            mred = 0;
            return mred
        end    
    end
return mred
end
end

reduce_term_relation_1 (generic function with 1 method)

The second relation is:

**Relation 2**: $x_i\cdot \Delta_{ij} = x_j \cdot\Delta_{ij} = x_i\cdot x_i$ for all $1\leq i< j \leq n$

This relation allows us to replace the variable $\Delta_{ij}$ with one variable $x_i$ or $x_j$. 

This is implemented by the following function:

In [3]:
function reduce_term_relation_2(m)
if m==0
    return m;
else
    mred = m;
    for i in 1:n
        if mred == 0
            return 0;        
        elseif exponent(mred,1,var_index(x[i]))>1
            mred = 0
            return mred
        elseif exponent(mred,1,var_index(x[i]))==1
            for j in i+1:n
                if mred == 0
                    return 0
                elseif exponent(mred,1,var_index(Delta[i,j]))>0
                    mred = divexact(mred,Delta[i,j])*x[j];
                end
            end
            for j in 1:i-1
                if mred == 0
                    return 0
                elseif exponent(mred,1,var_index(Delta[j,i]))>0
                    mred = divexact(mred,Delta[j,i])*x[j];
                    mred = reduce_term_relation_2(mred);
                end
            end
        end
    end
return mred
end
end

reduce_term_relation_2 (generic function with 1 method)

The third relation is

**Relation 3**: $\Delta_{ij}^3 = 0$ and $\Delta_{ij}^2 = -(2g-2)\cdot x_i \cdot x_j$ for all $1\leq i<j\leq n$

This relation allows us to replace a higher power of $\Delta_{ij}$ by zero or by a term in $x_i,x_j$. It is implemented by the following function:

In [4]:
function reduce_term_relation_3(m)
if m==0
    return m
else
    mred = m;
    # This block implements relation (3)
    for i in 1:n
        for j in i+1:n
            if exponent(mred,1,var_index(Delta[i,j]))>2
                mred=0;
                return mred
            elseif exponent(mred,1,var_index(Delta[i,j]))==2
                mred = -(2*g-2)*x[i]*x[j]*divexact(mred,(Delta[i,j]^2));
            end
        end
    end
return mred
end
end

reduce_term_relation_3 (generic function with 1 method)

These relations all together allow us to write any term in the $x_i$ and $\Delta_{ij}$ in the *reduced form*

$$ x_{i_1}\cdot x_{i_2} \cdot \dots \cdot x_{i_r} \cdot \Delta_{h_1k_1} \cdot \dots \Delta_{h_sk_s} 
$$ 
where $i_1,\dots,i_r$ and $(h_1k_1),\dots,(h_sk_s)$ are pairwise distinct and the two groups of indices are disjoint, i.e.
$$
\{ i_1,\dots,i_r \} \cap \{ h_1,k_1,\dotsc,h_s,k_s \} = \emptyset.  
$$
This is implemented by the following two functions, for a single term and for a polynomial:

In [5]:
# This function brings an arbitrary term into reduced form

function reduce_term(m)
    mred = m;
    mred = reduce_term_relation_1(mred);
    mred = reduce_term_relation_3(mred);
    mred = reduce_term_relation_2(mred);
return mred    
end

# This function takes a polynomial and brings each term into reduced form

function reduce_polynomial(F)
    Fred = 0;
    for i in 1:length(terms(F))
        Fred = Fred + reduce_term(term(F,i))
    end
return Fred
end

reduce_polynomial (generic function with 1 method)

Some easy examples are:

In [7]:
# Some examples

# We set the genus g of the curve C and the number of copies n in the Cartesian product C^n
n = 7
g = 3

# We define the polynomial ring that we use to describe the intersection product: it has variables x[1],...,x[n] and Delta[1,2],...,Delta[n-1,n]

RCart, x, Delta = polynomial_ring(QQ, "x" => (1:n), "Delta" => (1:n, 1:n));

#reduce_term(x[1]*x[3]*x[4]*Delta[1,5]*Delta[3,4]*Delta[4,6])
#reduce_term(x[1]*x[3]*x[4]*Delta[1,5]*Delta[3,4])
reduce_term(x[2]*Delta[1,2]*Delta[1,4])

x[1]*x[2]*x[4]

## Intersection numbers 

Once we have a monomial of degree $n$ in reduced form we can compute the corresponding intersection number via a graph-theoretic argument, following Lemma 4.2 of [APSW]. Let
$$ x_{i_1}\cdot x_{i_2} \cdot \dots \cdot x_{i_r} \cdot \Delta_{h_1k_1} \cdot \dots \Delta_{h_sk_s} 
$$ 
be a monomial in reduced form of degree $n$. Then let $\Gamma$ be the graph with vertex set $V=\{h_1,k_1,\dots,h_s,k_s\}$ and an edge between two vertices if and only $\Delta_{h_ik_i}$ appears in the above monomial. We look at the connected components of $\Gamma$: if each connected component has the same number of vertices and edges, then the intersection product is
$$ x_{i_1}\cdot x_{i_2} \cdot \dots \cdot x_{i_r} \cdot \Delta_{h_1k_1} \cdot \dots \Delta_{h_sk_s}  = (2-2g)^{\# \text{ conn. comp. }}
$$ 
otherwise, the intersection product is zero.

In [8]:
# The following function takes a term in reduced form of top degree n in C^n and gives the corresponding intersection number.
# Note that the polynomial should be in reduced form

function intersection_product_term_reduced(m)
    E = exponent_vector(m,1);
    E = E[n+1:length(E)];
    A = reshape(E,(n,n))+copy(transpose(reshape(E,(n,n))));
    G = Graphs.SimpleGraph(A);
    connG = Graphs.connected_components(G);
    intprod = coeff(m,1);
    for i in connG
        cg = Graphs.induced_subgraph(G,i)[1]
        if (Graphs.nv(cg)!=1) && (Graphs.nv(cg) != Graphs.ne(cg))
            return 0;
        elseif (Graphs.nv(cg)!=1) && (Graphs.nv(cg) == Graphs.ne(cg))
            intprod = intprod*(-(2g-2))
        end
    end
return intprod 
end

# We can now iterate this function over a polynomial of top degree n where all terms are in reduced form

function intersection_product_polynomial_reduced(F)
    intprod = 0
    for i in 1:length(terms(F))
        intprod = intprod + intersection_product_term_reduced(term(F,i)) 
    end
    return intprod
end



# We can now iterate this function over a polynomial of top degree n where all terms are in reduced form

function intersection_product_polynomial_reduced(F)
    intprod = 0
    for i in 1:length(terms(F))
        intprod = intprod + intersection_product_term_reduced(term(F,i)) 
    end
    return intprod
end

intersection_product_polynomial_reduced (generic function with 1 method)

# Example: Scorza correspondences

We apply these methods to some intersection problems related to Scorza correspondences, following Section 4.2 of [APSW]. Let $C$ be a smooth plane quartic curve of genus $3$, and let $\eta$ be an even theta characteristic on it. The Scorza correspondence is the divisor in $C\times C$ given by
$$ S(\eta) = \{(x,y) \,|\, h^0(C,\eta(x-y)) > 0\} $$
Its class inside $C\times C$ is given by
$$ [S(\eta)] = 2x_1 + 2x_2 + \Delta $$
Inside the product $C^n$ we can consider the pullbacks of the Scorza correspondence $S_{ij} = \operatorname{pr}_{i,j}^*S(\eta)$. In particular, following Lemma 4.5 of [APSW], we expect that on $C^7$ we have
$$ [S_{12}]\cdot [S_{23}]\cdot [S_{34}]\cdot [S_{45}]\cdot [S_{56}]\cdot [S_{67}]\cdot [S_{17}] = 4638$$
We can compute this explicitly as follows.

In [12]:
# We set the genus g of the curve C and the number of copies n in the Cartesian product C^n
n = 7
g = 3

# We define the polynomial ring that we use to describe the intersection product: it has variables x[1],...,x[n] and Delta[1,2],...,Delta[n-1,n]

RCart, x, Delta = polynomial_ring(QQ, "x" => (1:n), "Delta" => (1:n, 1:n));


# We compute the reduced form of the product of Scorza correspondences: 

S12 = 2*x[1]+2*x[2]+Delta[1,2];
S23 = 2*x[2]+2*x[3]+Delta[2,3];
S34 = 2*x[3]+2*x[4]+Delta[3,4];
S45 = 2*x[4]+2*x[5]+Delta[4,5];
S56 = 2*x[5]+2*x[6]+Delta[5,6];
S67 = 2*x[6]+2*x[7]+Delta[6,7];
S17 = 2*x[1]+2*x[7]+Delta[1,7];

F = S12*S23*S34*S45*S56*S67*S17;

# The expression for the reduced polynomial is quite easy

Fred = reduce_polynomial(F)

4372*x[1]*x[2]*x[3]*x[4]*x[5]*x[6]*x[7] + Delta[1, 2]*Delta[2, 3]*Delta[3, 4]*Delta[4, 5]*Delta[5, 6]*Delta[1, 7]*Delta[6, 7]

In [11]:
# We expect now to obtain 4368

intersection_product_polynomial_reduced(Fred)

4368